# Data manipulation using .groupby()

Bab ini menjelaskan fungsi `groupby()` dan bagaimana kita dapat menggunakannya untuk mengubah nilai yang ada, mengganti nilai yang hilang dan menerapkan fungsi kompleks berdasarkan kelompok.

## Data transformation using `.groupby()` and `.transform()`

### The restaurant dataset

In [9]:
import pandas as pd
import numpy as np
import time

In [79]:
restaurant = pd.read_csv("datasets/restaurant_data.csv")
restaurant.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [5]:
restaurant_grouped = restaurant.groupby("smoker")
restaurant_grouped.count()

,total_bill,tip,sex,day,time,size
smoker,,,,,,
No,151,151,151,151,151,151
Yes,93,93,93,93,93,93


### Data transformation

In [6]:
zscore = lambda x: (x - x.mean() ) / x.std()

In [7]:
restaurant_grouped = restaurant.groupby("time")
restaurant_transformed = restaurant_grouped.transform(zscore)
restaurant_transformed.head()

,total_bill,tip,size
0,-0.416446,-1.457045,-0.692873
1,-1.143855,-1.004475,0.405737
2,0.023282,0.276645,0.405737
3,0.315339,0.144355,-0.692873
4,0.414880,0.353234,1.504347


### Comparison with native methods

In [10]:
time_start = time.time()
restaurant.groupby('sex').transform(zscore)
time_a = time.time() - time_start
print("Time using .groupby(): {} seconds".format(time_a))

Time using .groupby(): 0.023209571838378906 seconds


In [11]:
time_start = time.time()
mean_female = restaurant.groupby('sex').mean()['total_bill']['Female']
mean_male = restaurant.groupby('sex').mean()['total_bill']['Male']
std_female = restaurant.groupby('sex').std()['total_bill']['Female']
std_male = restaurant.groupby('sex').std()['total_bill']['Male']

for i in range(len(restaurant)):
    if restaurant.iloc[i][2] == 'Female':
        restaurant.iloc[i][0] = (restaurant.iloc[i][0] - mean_female)/std_female
    else:
        restaurant.iloc[i][0] = (restaurant.iloc[i][0] - mean_male)/std_male

time_b = time.time() - time_start
print("Time using native Python: {} seconds".format(time_b))

Time using .groupby(): 0.17777132987976074 seconds


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [23]:
# Function different time
def diff_time(a, b):
    if a > b:
        print("Difference in speed: {} %".format((a - b) / b * 100))
    if b > a:
        print("Difference in speed: {} %".format((b - a) / a * 100))

In [24]:
diff_time(time_a, time_b)

Difference in speed: 665.939721411842 %


### The min-max normalization using `.transform()`

Operasi yang sangat umum adalah *min-max normalization*. Ini terdiri dari menskalakan kembali nilai bunga dengan mengurangi nilai minimum dan membagi hasilnya dengan selisih antara nilai maksimum dan nilai minimum. Misalnya, untuk menghitung kembali data berat badan siswa yang berkisar dari 160 pon hingga 200 pon, Anda kurangi 160 dari masing-masing bobot siswa dan bagilah hasilnya dengan 40 (200 - 160).

Anda akan mendefinisikan dan menerapkan normalisasi min-max ke semua variabel numerik dalam data restoran. Pertama-tama Anda akan mengelompokkan entri pada saat makan berlangsung (Makan Siang atau Makan Malam) dan kemudian menerapkan normalisasi untuk masing-masing kelompok secara terpisah.

In [26]:
# Define the min-max transformation
min_max_tr = lambda x: (x - x.min()) / (x.max() - x.min())

# Group the data according to the time
restaurant_grouped = restaurant.groupby('time')

# Apply the transformation
restaurant_min_max_group = restaurant_grouped.transform(min_max_tr)
restaurant_min_max_group.head()

,total_bill,tip,size
0,0.291579,0.001111,0.2
1,0.152283,0.073333,0.4
2,0.375786,0.277778,0.4
3,0.431713,0.256667,0.2
4,0.450775,0.290000,0.6


**Catatan** : Sekarang Anda dapat menggunakan fungsi transformasi dalam transformasi apa pun yang dapat Anda tetapkan!

### Transforming values to probabilities

Dalam latihan ini, kami akan menerapkan fungsi distribusi probabilitas ke Pandas DataFrame dengan parameter group dan mengubah variabel tip menjadi probabilitas.

Anda akan menerapkan transformasi distribusi eksponensial ke ukuran setiap tabel dalam dataset, setelah mengelompokkan data sesuai dengan waktu hari makan berlangsung. Ingatlah untuk menggunakan rata-rata setiap kelompok untuk nilai λ.

Dalam Python, Anda bisa menggunakan eksponensial sebagai `np.exp()` dari NumPy dan nilai rata-rata sebagai `.mean()`.

In [27]:
# Define the exponential transformation
exp_tr = lambda x: np.exp(-x.mean() * x) * x.mean()

# Group the data according to the time
restaurant_grouped = restaurant.groupby('time')

# Apply the transformation
restaurant_exp_group = restaurant_grouped['tip'].transform(exp_tr)
print(restaurant_exp_group.head())

0    0.135141
1    0.017986
2    0.000060
3    0.000108
4    0.000042
Name: tip, dtype: float64


### Validation of normalization

Untuk latihan ini, kita akan melakukan normalisasi z-score dan memverifikasi bahwa itu dilakukan dengan benar.

Karakteristik yang berbeda dari nilai yang dinormalisasi adalah bahwa nilai rata-rata sama dengan nol dan standar deviasi sama dengan satu.

Setelah Anda menerapkan transformasi normalisasi, Anda dapat mengelompokkan lagi pada variabel yang sama, dan kemudian memeriksa mean dan standar deviasi masing-masing grup.

In [32]:
poker_hands = pd.read_csv("datasets/poker_hand.csv")
poker_grouped = poker_hands.groupby("Class")

In [34]:
zscore = lambda x: (x - x.mean()) / x.std()

# Apply the transformation
poker_trans = poker_grouped.transform(zscore)
poker_trans.head()

,S1,R1,S2,R2,S3,R3,S4,R4,S5,R5
0,-1.380537,0.270364,-1.380537,-0.730297,-1.380537,0.631224,-1.380537,0.350823,-1.380537,-0.724286
1,-0.613572,0.495666,-0.613572,1.095445,-0.613572,0.039451,-0.613572,0.350823,-0.613572,-0.724286
2,0.153393,0.720969,0.153393,-0.730297,0.153393,0.631224,0.153393,-1.403293,0.153393,-0.724286
3,0.920358,0.270364,0.920358,-0.730297,0.920358,-1.735866,0.920358,1.227881,0.920358,1.267500
4,0.920358,-1.757363,0.920358,1.095445,0.920358,0.433966,0.920358,-0.526235,0.920358,0.905357


In [35]:
zscore = lambda x: (x - x.mean()) / x.std()

# Apply the transformation
poker_trans = poker_grouped.transform(zscore)

# Re-group the grouped object and print each group's means and standard deviation
poker_regrouped = poker_trans.groupby(poker_hands['Class'])

print(np.round(poker_regrouped.mean(), 3))
print(poker_regrouped.std())

        S1   R1   S2   R2   S3   R3   S4   R4   S5   R5
Class                                                  
0     -0.0 -0.0  0.0 -0.0  0.0  0.0  0.0  0.0 -0.0  0.0
1      0.0  0.0 -0.0  0.0 -0.0  0.0  0.0  0.0 -0.0  0.0
2     -0.0 -0.0  0.0 -0.0 -0.0  0.0  0.0 -0.0 -0.0  0.0
3      0.0  0.0  0.0 -0.0 -0.0 -0.0 -0.0 -0.0  0.0 -0.0
4     -0.0 -0.0 -0.0 -0.0  0.0 -0.0 -0.0  0.0  0.0  0.0
5     -0.0 -0.0 -0.0  0.0 -0.0  0.0 -0.0 -0.0 -0.0  0.0
6     -0.0 -0.0 -0.0  0.0  0.0 -0.0  0.0  0.0 -0.0  0.0
7      0.0 -0.0 -0.0  0.0 -0.0  0.0  0.0 -0.0 -0.0 -0.0
8     -0.0  0.0 -0.0  0.0 -0.0  0.0 -0.0  0.0 -0.0 -0.0
9      0.0 -0.0  0.0 -0.0  0.0 -0.0  0.0  0.0  0.0 -0.0
        S1   R1   S2   R2   S3   R3   S4   R4   S5   R5
Class                                                  
0      1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
1      1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
2      1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
3      1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1

**Catatan** : Sekarang Anda tahu bahwa normalisasi dilakukan dengan benar, karena rata-rata dalam setiap grup yang dinormalisasi adalah 0 dan standar deviasi adalah 1!

### When to use transform()?

Fungsi `.transform()` menerapkan fungsi ke semua anggota setiap grup. Manakah dari transformasi berikut ini yang akan menghasilkan hasil yang sama di seluruh dataset terlepas dari pengelompokan?

* `lambda x: np.random.randint(0,10)`

## Missing value imputation using `transform()`

### Practice: Identifying missing values

Langkah pertama sebelum imputasi nilai yang hilang adalah untuk mengidentifikasi apakah ada nilai yang hilang dalam data, dan jika demikian, dari kelompok mana mereka muncul.

In [80]:
# Prepare data with missing values
restaurant_nan = restaurant.copy()
restaurant_nan.iloc[::4, 1] = np.nan

In [81]:
# Group both objects according to smoke condition
restaurant_nan_grouped = restaurant_nan.groupby('smoker')

# Store the number of present values
restaurant_nan_nval = restaurant_nan_grouped['tip'].count()

# Print the group-wise missing entries
print(restaurant_nan_grouped['total_bill'].count() - restaurant_nan_nval)

smoker
No     36
Yes    25
dtype: int64


### Missing value imputation

Karena sebagian besar data real-world berisi entri yang hilang, mengganti entri ini dengan nilai yang masuk akal dapat meningkatkan wawasan yang bisa Anda dapatkan dari data.

Dalam dataset restoran, kolom `"total_bill"` memiliki beberapa entri yang hilang, artinya Anda belum mencatat berapa banyak tabel telah membayar. Tugas Anda dalam latihan ini adalah mengganti entri yang hilang dengan nilai median dari jumlah yang dibayarkan, sesuai dengan apakah entri dicatat pada makan siang atau makan malam (variabel waktu).

In [97]:
# Prepare data with missing values
restaurant_data = restaurant.copy()
restaurant_data.iloc[::4, 0] = np.nan

In [98]:
restaurant_data.head()

,total_bill,tip,sex,smoker,day,time,size
0,NaN,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,NaN,3.61,Female,No,Sun,Dinner,4


In [99]:
# Define the lambda function
missing_trans = lambda x: x.fillna(x.median())

# Group the data according to time
restaurant_grouped = restaurant_data.groupby('time')

# Apply the transformation
restaurant_impute = restaurant_grouped.transform(missing_trans)
restaurant_impute.head()

,total_bill,tip,size
0,18.39,1.01,2
1,10.34,1.66,3
2,21.01,3.50,3
3,23.68,3.31,2
4,18.39,3.61,4


## Data filtration using the `filter()` function

### When to use filtration?

Saat menerapkan fungsi `filter()` pada objek yang dikelompokkan, apa yang dapat Anda gunakan sebagai kriteria untuk memfilter?

* Jumlah nilai yang hilang dari suatu fitur.
* Angka rata-rata fitur.
* Rata-rata numerik lebih dari satu fitur.

### Data filtration

Anda mungkin perlu memfilter data Anda karena berbagai alasan.

Dalam latihan ini, Anda akan menggunakan pemfilteran untuk memilih bagian tertentu dari DataFrame:

* dengan jumlah entri yang direkam di setiap hari dalam seminggu
* dengan jumlah rata-rata uang yang dibayarkan pelanggan ke restoran setiap hari dalam seminggu

In [100]:
# Filter the days where the count of total_bill is greater than $40
total_bill_40 = restaurant.groupby('day').filter(lambda x: x['total_bill'].count() > 40)

# Print the number of tables where total_bill is greater than $40
print('Number of tables where total_bill is greater than $40:', total_bill_40.shape[0])

Number of tables where total_bill is greater than $40: 225


In [103]:
# Select only the entries that have a mean total_bill greater than $20
total_bill_20 = total_bill_40.groupby('day').filter(lambda x : x['total_bill'].mean() > 20)

# Print days of the week that have a mean total_bill greater than $20
print('Days of the week that have a mean total_bill greater than $20:', total_bill_20.day.unique())

Days of the week that have a mean total_bill greater than $20: ['Sun' 'Sat']


**Catatan** : Anda baru saja membantu sebuah restoran menyesuaikan waktu pembukaan mereka sesuai dengan laba. Tidak mengherankan, sepertinya akhir pekan adalah waktu yang paling menguntungkan dalam seminggu!

## What you have learned

* Mengapa dan bagaimana mengatur waktu operasi
* Pilih baris dan kolom yang ditargetkan secara efisien
* Pilih baris dan kolom acak secara efisien
* Ganti nilai-nilai DataFrame secara efisien menggunakan `replace()`
  * Ganti beberapa nilai menggunakan lists
  * Ganti beberapa nilai menggunakan dictionaries
* Iterate pada DataFrame menggunakan fungsi `.iterrows()`
* Iterate pada DataFrame menggunakan fungsi `.apply()`
* Iterate pada DataFrame menggunakan optimasi pandas
* Iterate pada DataFrame menggunakan optimasi numpy
* Perbandingan fungsi `groupby()` dibandingkan dengan kode python asli
  * Saat mengubah data berdasarkan group-wise
  * Saat memasukkan nilai yang hilang berdasarkan group-wise/kelompok
  * Ketika menyaring kelompok dengan karakteristik khusus